# Arbeitspaket (AP) 2: Management & Nutzung Temporale Daten

### Persönliche Angaben (bitte ergänzen)

<table>
  <tr>
    <td>Vorname:</td>
    <td></td>
  </tr>
  <tr>
    <td>Nachname:</td>
    <td></td>
  </tr>
  <tr>
    <td>Immatrikulationsnummer:</td>
    <td></td>
  </tr>
  <tr>
    <td>Modul:</td>
    <td>Data Science</td>
  </tr>
  <tr>
    <td>Prüfungsdatum / Raum / Zeit:</td>
    <td>06.10.2025 / Raum: MU O2.001 / 8:00 – 11:45</td>
  </tr>
  <tr>
    <td>Erlaubte Hilfsmittel:</td>
    <td>w.MA.XX.DS.25HS (Data Science)<br>Open Book, Eigener Computer, Internet-Zugang</td>
  </tr>
  <tr>
  <td>Nicht erlaubt:</td>
  <td>Nicht erlaubt ist der Einsatz beliebiger Formen von generativer KI (z.B. Copilot, ChatGPT) <br> sowie beliebige Formen von Kommunikation oder Kollaboration mit anderen Menschen.</td>
</tr>
</table>

## Bewertungskriterien

### <b style="color: gray;">(max. erreichbare Punkte: 40)</b>

<table>
  <thead>
    <tr>
      <th>Kategorie</th>
      <th>Beschreibung</th>
      <th>Punkteverteilung</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Code nicht lauffähig oder Ergebnisse nicht sinnvoll</td>
      <td>Der Code enthält Fehler, die verhindern, dass er ausgeführt werden kann (z.B. Syntaxfehler) oder es werden Ergebnisse ausgegeben, welche nicht zur Fragestellung passen.</td>
      <td>0 Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit gravierenden Mängeln</td>
      <td>Der Code läuft, aber die Ergebnisse sind aufgrund wesentlicher Fehler unvollständig (z.B. fehlende Joins, gravierende Fehler in SQL-Abfragen). Nur geringer Fortschritt erkennbar.</td>
      <td>25% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit mittleren Mängeln</td>
      <td>Der Code läuft und liefert teilweise korrekte Ergebnisse, aber es gibt grössere Fehler (z.B. fehlende Spalten, unvollständige SQL-Abfragen). Die Ergebnisse sind nachvollziehbar, aber unvollständig oder ungenau.</td>
      <td>50% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit minimalen Mängeln</td>
      <td>Der Code läuft und liefert ein weitgehend korrektes Ergebnis, aber kleinere Fehler (z.B. falsche oder fehlende Sortierung, Rundung von Werten falsch) beeinträchtigen die Vollständigkeit des Ergebnisses.</td>
      <td>75% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig und korrekt</td>
      <td>Der Code läuft einwandfrei und liefert das korrekte Ergebnis ohne Mängel.</td>
      <td>100% der max. erreichbaren Punkte</td>
    </tr>
  </tbody>
</table>



## <b>Vorbereitung (Dieser Teil wird <u>nicht</u> bewertet!)</b>

#### <b>Beschreibung des Datensatzes `synthetic_ecommerce_orders.csv`</b>

- **Inhalt:** Synthetische E-Commerce-Bestellereignisse (April–Juni 2024)
- **Schema:**
  - `time` (lokale Zeit, Europe/Zurich, Format `YYYY-MM-DD HH:MM:SS`)
  - `customer_id` (int)
  - `product_id` (int)
  - `price` (float, EUR)
  - `discount` (float, EUR)
  - `status` (category: `placed`, `paid`, `cancelled`, `refunded`)
- **Hinweise zur Nutzung:**
  - Für **Abfragen** mit Tinyflux Zeitstempel nach **UTC** konvertieren.
  - `price - discount` kann als **Nettoerlös** interpretiert werden (v. a. für `paid`).
  - Leichte Tages-/Wochenmuster (mehr Aktivität 09:00–21:00, werktags).
- **Ziel:** Einlesen, in Tinyflux schreiben (Fields: `price`, `discount`; Tags: `customer_id`, `product_id`, `status`, optional `channel`), Zeitfenster-Abfragen und Aggregationen.


#### <b>1.) Python Libraries und Settings importieren:</b>

In [ ]:
# Libraries
import os
import pandas as pd
import csv
from tinyflux import TinyFlux, Point, FieldQuery, TagQuery, TimeQuery
from datetime import datetime, timezone, timedelta

# Settings
import warnings
warnings.filterwarnings("ignore")

# Abkürzungen für Query-Typen
time = TimeQuery()
tags = TagQuery()
field = FieldQuery()

#### <b>2.) Funktion für die Erstellung eines Dataframes aus Query-Resultaten in den Arbeitsspeicher laden:</b>

In [ ]:
# Define the function to transform the list of Point objects into a DataFrame
def points_to_dataframe(points_list):
    data = []
    
    # Extract the relevant information
    for point in points_list:
        data.append({
            "time": point.time,  # Access time attribute
            "customer_id": point.tags['customer_id'],  # Access customer_id from tags dictionary
            "product_id": point.tags['product_id'],  # Access product_id from tags dictionary
            "status": point.tags['status'],  # Access status from tags dictionary
            "price": point.fields['price'],  # Access price from fields dictionary
            "discount": point.fields['discount']  # Access discount from fields dictionary
        })

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Convert time to datetime
    df['time'] = pd.to_datetime(df['time'])

    return df

#### <b>3.) Starten Sie eine GitHub Codespaces Instanz auf Basis Ihres Forks des folgenden GitHub Repositories:</b>

##### GitHub-Repository: https://github.com/MariaPelli/tinyflux


#### <b>4.) Erstellen Sie mit Tinyflux eine Datenbank mit dem Namen "ecommerce.db".</b>

In [ ]:
# Erstellung der Tinyflux-Datenbank
db = TinyFlux("ecommerce.db")

## <b>Aufgaben: Einlesen der Daten in die Datenbank (Dieser Teil wird bewertet!)</b>

#### <b>Aufgabe (1): Lesen Sie die CSV-Datei 'synthetic_ecommerce_transactions.csv' in ein Dataframe mit dem Namen "df" ein, zeigen Sie die ersten 10 Records und geben deskriptive Statistiken des Dataframes aus.</b>

<b>Details zur Aufgabenstellung:</b>

- Das Laden der CSV-Datei kann mittels der Pandas-Funktion "read_csv" (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) erfolgen
- Achten Sie darauf, den Zeitstempel mittels der Pandas-Funktion __df['timestamp']= pd.to_datetime(df['timestamp'], format='mixed')__ auf das Format datetime zu setzen (https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)
- Die ersten 10 Records können mittels der Pandas-Funktion "df.head()" wiedergegeben werden (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html) 
- Deskriptive Statistiken können mittels Pandas-Funktion "df.describe()" (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) wiedergegeben werden. Geben Sie diese für quantitative und kategoriale Variablen aus, indem Sie den Parameter include auf 'all' setzen.

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [ ]:
# CSV-Datei in Dataframe laden


# Das Format des Zeitstempels in Pandas setzen


In [ ]:
# Erste 10 Records des Dataframes anzeigen


In [ ]:
# Deskriptive Statistiken des Dataframes


#### <b>Aufgabe (2): Schreiben Sie den Inhalt der CSV-Datei 'ecommerce.csv' in die erstellte Tinyflux-Datenbank "ecommerce.db" und weisen sie Time, Fields und Tags zu.</b>

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [ ]:
# Tinyflux Datenbank leeren
os.path.exists('ecommerce.db') and open('ecommerce.db','w').close()

# Einlesen der CSV-Datei in Tinyflux


#### <b>Aufgabe (3): Die ersten drei Käufe erhalten am 7.7.2024 10% auf den am 1.4.2024 bezahlten Preis (ohne Berücksichtigung des Discounts) zurückerstattet. Fügen Sie die drei Transaktionen manuell in die Datenbank ein und weisen Sie Time, Fields und Tags gleichermassen zu, wie beim Inhalt der CSV-Datei.

</b>

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [ ]:
# Berechnung der Rückerstattung am 7.7.2024

v1 = 46.79 * 0.1
print('Rückerstattung 1. Kauf:', v1)
v2 = 22.37 * 0.1
print('Rückerstattung 2. Kauf:', v2)
v3 = 38.76 * 0.1
print('Rückerstattung 3. Kauf:', v3)

| time | customer_id | product_id | price | discount | status |
| --- | --- | --- | --- | --- | --- |
| 2024-07-07 00:00:00 | 12828 | 1114 | 4.679 | 0.00 | refunded |
| 2024-07-07 00:00:00 | 12313 | 1414 | 2.237 | 0.00 | refunded |
| 2024-07-07 00:00:00 | 12117 | 1242 | 3.876 | 0.00 | refunded |

In [ ]:
# Einlesen von manuellen Records


<div>

#### <b> Hinweis! 

#### Falls etwas mit der Erstellung der Datenbank nicht funktioniert, verwenden Sie für die folgenden Aufgaben die Tinyflux Datenbank 'ecommerce_fallback.db'. Gehen Sie davon aus, dass diese unter denselben, obigen Bedingungen erstellt wurde.</b>
    
#### <b>Erstellen der Verbindung mit der Tinyflux-Datenbank mit dem Namen "ecommerce_fallback.db":</b>
</div>

In [ ]:
# Erstellung der Tinyflux-Datenbank (wird bereitgestellt)
db = TinyFlux("ecommerce_fallback.db")

## <b>Aufgaben: Abfragen (Queries) (Dieser Teil wird bewertet!)</b>

<b >Hinweise zu den folgenden Aufgabenstellungen:</b>
<ul>
  <li>Erstellen Sie je Aufgabe eine Time-, Field- oder Tag Abfrage bzw. eine Kombination von diesen und integrieren Sie diese in Ihren Python Code.</li>
  <li>Speichern Sie je Aufgabe die Ergebnistabelle mittels der zu Beginn dieses Notebooks bereitgestellten Funktion <b>points_to_dataframe</b> in einem DataFrame und stellen Sie diesen bzw. dessen Eigenschaften ganz oder in Teilen (z.B. erste 5 Zeilen) dar.</li>
</ul>

#### <b>Aufgabe (4): Schreiben Sie alle Bestellungen in ein Dataframe, die bis und mit dem 4.4.2024 getätigt wurden und geben Sie die ersten 10 Records dieses Dataframes aus</b>
<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [ ]:
# Definieren der Query


# Datenabfrage mittels der definierten Query


# Erstellung eines Dataframes aus den Query-Resultaten und Ausgabe der ersten 10 Records



#### <b>Aufgabe (5): Ermitteln Sie über den gesamten Datensatz die Anzahl Transaktionen pro Monat</b>

<b>Details zur Aufgabenstellung:</b>
- Die Anzahl Transaktionen kann mittels len(df_result), angewendet auf das Dataframe oder die Rohausgabe des Queries, herausgegeben werden

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [ ]:
# Definieren der Queries


# Datenabfrage mittels der definierten Queries


# Erstellung eines Dataframes aus den Query-Resultaten


# Ausgabe der aggregierten Resultate



#### <b>Aufgabe (6): Ermitteln Sie den Median aller Discount-Beträge pro Status und Monat</b>

<b>Details zur Aufgabenstellung:</b>
- Alle im Datensatz vorhandenen Status können Sie mittels der Pandas-Funktion "df.attributname.unique()" (https://pandas.pydata.org/docs/reference/api/pandas.unique.html) ermitteln
- Den Durchschnitt über eine Spalte können Sie mittels der Pandas-Funktion df[['spaltenname']].mean() https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html ermitteln

<b style="color: gray;">(max. erreichbare Punkte: 12)</b>

In [ ]:
df.status.unique()

In [ ]:
# Definieren der Time-Queries


# Definieren der Tag-Queries


# Datenabfrage mittels der definierten Queries


# Erstellung eines Dataframes aus den Query-Resultaten


# Ausgabe der aggregierten Resultate

# - paid, April

# - placed, April

# - refunded, April

# - cancelled, April



# - paid, Mai

# - placed, Mai

# - refunded, Mai

# - cancelled, Mai



# - paid, Juni

# - placed, Juni

# - refunded, Juni

# - cancelled, Juni


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('IP Address:', socket.gethostbyname(socket.gethostname()))
print('-----------------------------------')